In [2]:
import pandas as pd

from scipy.optimize import fsolve
from openpyxl import load_workbook

import numpy as np
import uncertainties as uc
from uncertainties import ufloat
from uncertainties import unumpy

pd.options.display.float_format ='{:,.1f}'.format
import matplotlib.pyplot as plt
%matplotlib inline  
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pylab as pylab

from scipy import integrate
import scipy.stats as stats

from sklearn.linear_model import LinearRegression

# Mouse comparison 

We use data gathered from close to 20 cell types to make a regression connecting the the lifespan of cell in a mouse body to its lifespan in a human body.
We create a function that gets an array of lifespans in mouse and return their matching predicted lifespan in human, and its uncertainty, as a multiplication factor, relating to the standard error of the prediction 


In [3]:
#reading the data regarding the lifespan of mouce and human cells
mus_hu = pd.read_excel('Mouse_comparison_data.xlsx','lifespan_data',index_col=0,usecols = range(9))
# mus_hu = mus_hu.drop(['Tcell CD4+Memory','Tcell CD8+Naïve','Tcell CD8+Memory']) #,'Tcell CD4+Naïve'


#adding columns containing the log for a regression in logspace (power low)
mus_hu['log mouse'] = np.log10(mus_hu['mouse lifespan'])
mus_hu['log human'] = np.log10(mus_hu['human lifespan'])

mus_hu.head()

,mouse lifespan,mouse SD,human lifespan,human SD,source for mouse,source for human,comment,colors,log mouse,log human
cell type,,,,,,,,,,
erythrocytes,54.0,2.9,119.3,3.3,"Landaw et al., 1970, van Putten and Croon, 1958",NaN,average of their results for different mice,"rgb(156, 0, 0)",1.7,2.1
neutrophils,3.0,nan,6.6,0.1,"Pillay at el., 2010","Lahoz-beneytez et al., 2015",NaN,"rgb(183,70,70)",0.5,0.8
epidermal cells,9.5,1.0,30.0,0.0,"Potten et al., 1987","Halprin, 1972, Bergstresser and Taylor, 1977,...",calculated as 5 days cell cycle in basal layer...,"rgb(217, 93, 41)",1.0,1.5
endothelial cells,300.0,6.6,"2,281.2","1,250.0","Hobson and Denekamp, 1984","Bergmann et al., 2015",NaN,"rgb(205, 160, 70)",2.5,3.4
small intestinal epithelial cells,2.8,1.0,2.2,0.9,"Darwich et al., 2014","Darwich et al., 2014",NaN,"rgb(138, 94, 49)",0.4,0.3


Creating a power low model by using a regression in log-log space.

We define some functions to estimate the uncertainty in the prediction using the statistics of the regression, following the formula:
$$SE =Sres\sqrt{\frac{1}{n}+\frac{(x_{0}-\bar{x})^{2}}{SSx}}$$
where: $SE$ is the standard error of the predicted value.
$$SSx =\sum{(x-\bar{x})^{2}}$$
$$Sres = \sqrt{ \frac{1}{n-2}(\sum{(y-\bar{y})^{2}} - \frac{[\sum{(x-\bar{x})(y-\bar{y})}]^{2}}{\sum{(x-\bar{x})^{2}}})} $$


In [4]:
#fitting a linear regression to the log log data
model = LinearRegression().fit(mus_hu['log mouse'].values.reshape(-1, 1),mus_hu['log human'].values.reshape(-1, 1))
r_sq= model.score(mus_hu['log mouse'].values.reshape(-1, 1), mus_hu['log human'].values.reshape(-1, 1))


#defining some helpfull functions:

# function that compute the sum of squares of the distances from the mean for an array input
def SSQ(X):  
    return ((X-X.mean())**2).sum()

# function that compute the sum of squares of the distances from the mean for an array input
def Sxy(X,Y):  
    return np.dot(X-X.mean(),Y-Y.mean())  

# function that compute Sres for the input X,Y arrays 
def STEYX(X,Y):
    n=X.size
    return np.sqrt((1/(n-2))*(SSQ(Y)-Sxy(X,Y)**2/SSQ(X)))

#function that compute the linear standard error for the predicted value of a given x0.
#here x0 is given in the linear space and the result is in the log space
#the computation follow the formula that was given, by using the defined functions
def linSE(X,Y,x0):
#     moving to the log space
    xlog= np.log10(x0)
    n=X.size
    
    Sres = STEYX(X,Y)
    SSx = SSQ(X)
    Xmean = xlog.mean()
#     following the given formula
    return Sres*np.sqrt(1/n + (xlog-Xmean)**2/SSx)


# function that get an 1xn array of lifespans for cell in mouce and give their matching lifepsan in human
# the function return 2xn array where the first entry for each is thhe lifespan in human and the second entry is for the multiplcation factor of SE
def PowerPred(xarray):
    vals = (10**model.intercept_)*(xarray**model.coef_[0])  #using the coefficeints of the regression converted to power low formula
    #using the lineSE function to compute the uncertainty in the logspace and taking the power of 10 to conclude for the lifespan uncertainty
    unc_fac = 10**linSE(mus_hu['log mouse'].values,mus_hu['log human'].values,xarray) 
     #using numpy stack to put the arrays together
    return np.stack((vals,unc_fac), axis=1)



## Graphical illustration
Plotting the data for comparison between lifespan in humans and mice.
Using the function defined by the regression to show it and its 95% confidecr interval uncertainty.

The plotting of the graph is silenced so it won't appear when imported from other notebooks.

In [5]:
#pharsing the colors in RGB format from excel (used there in the form suitable for Voroni diagram)
mus_hu['R'] = pd.to_numeric([(x.split(',')[0]).split('(')[1] for x in mus_hu['colors']])
mus_hu['G'] = pd.to_numeric([x.split(',')[1] for x in mus_hu['colors']])
mus_hu['B'] = pd.to_numeric([(x.split(',')[2]).split(')')[0] for x in mus_hu['colors']])

In [6]:
Conf= 0.95  #defining the 1-alpha for the confidence interval

#computing the relevant critical t from the n-2 degree of freedom and the two way t inverse distribution, for the given confidence.
t_crit = stats.t.ppf(1-(1-Conf)/2, mus_hu['log mouse'].values.size-2)

# using linespace for the plot 
xspace = (10**np.linspace(-1, 4,25)).reshape(-1,1)  #to use x in logspace
ypred = PowerPred(xspace)  #using the prediction made using the function  

#taking the first entry as the values
value = ypred[:,0]

#computing the factor of uncertainty for the given confidence interval
CI_fac = ypred[:,1]**t_crit

#using the factor to get the lower and upper bound for the estimates.
y_low = value/CI_fac
y_high = value*CI_fac


#defining the parameters of the graph illustration
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (12,8),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)



#defining the colorsfor the plot as RGB (normalized for 0..1) 
colors = np.array([mus_hu['R'],mus_hu['G'],mus_hu['B']]).T/255
                   

#module for plotting the results is silenced for used in other notebooks.    
    
# plt.scatter(mus_hu['mouse lifespan'],mus_hu['human lifespan'],marker='o',s=100,c=colors)

# for cell in mus_hu.index:
#     plt.annotate(cell, (mus_hu.loc[cell,'mouse lifespan']*1.2, mus_hu.loc[cell,'human lifespan']*0.80))
    

# plt.plot(xspace,value,'b')
# plt.plot(xspace,y_low,'c--')
# plt.plot(xspace,y_high,'c--')

# plt.plot(xspace,xspace,'k-.') #1 to 1 ratio
# # plt.plot(xspace,40*xspace,'g-.') # 40 to 1 ratio


# plt.yscale('log')
# plt.xscale('log')

# plt.xlabel('mouse cell lifespan [d]')
# plt.ylabel('human cell lifespan [d]')
# plt.grid()



## Uncertainty of prediction for variable with error
When the mouse lifespan istself has some uncertainty factor

for a linear regression we get the connection:

$$\Delta y_{0} =\sqrt{{SE(x_{0})}^{2}+ {b}^{2}{\Delta x_{0} }^{2}}$$
$$y_{0} =bx_{0}+a$$
$$SE(x_{0}) =Sres\sqrt{\frac{1}{n}+\frac{(x_{0}-\bar{x})^{2}}{SSx}}$$

and so in our case the linear regressino is for the log log space, hence the connection between the factors of uncertainty $f_{y}$ and $f_{x}$
$$f_{y} =10^{\sqrt{{SE}^{2}+ {b}^{2}{log_{10}(f_{x}) }^{2}}}$$



In [7]:
#this function get a list of tuples that contains in their first entry the lifespan in mice [in days] and the second entry contain is uncertainty as a multiplication factor

def Pred_With_Unc(vals_n_unc):
    mo_vals = np.array([v for v,un in vals_n_unc])  #extracting the values and uncertainties from the input
    mo_unc_fac = np.array([un for v,un in vals_n_unc])
    
    hu_vals = (10**model.intercept_)*(mo_vals**model.coef_[0])  #using the coefficeints of the regression converted to power low formula
    
    #using the lineSE function to compute the prediction  uncertainty in the logspace
    #adding the uncertainty factor in mouse and taking the power of 10 to conclude for the lifespan uncertainty
   
    hu_unc_fac = 10**(np.sqrt(linSE(mus_hu['log mouse'].values,mus_hu['log human'].values,mo_vals)**2 + np.log10(mo_unc_fac)**2))
     #using numpy stack to put the arrays together
    return np.stack((hu_vals,hu_unc_fac), axis=1)



In [8]:
vals_n_unc = [(10,2), (30,1.2)]

Pred_With_Unc([(0.85,1)])

array([[1.32557655, 1.28749171]])

In [9]:
Pred_With_Unc([(300,1.4)])

array([[2.31684525e+03, 1.52317265e+00]])

In [10]:
Pred_With_Unc(vals_n_unc)

array([[ 30.54545276,   2.0965085 ],
       [123.64731192,   1.37366006]])